In [1]:
using Revise

In [2]:
using Boolean

In [3]:
init_logic(3)
println(simplifyLogic(Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)")))

x2 * (x3 * (x1 + x2))


In [4]:
create_bool_rep("z1 + z2")

Formula    = z1 + z2
Variable   = z
Size       = 3
Bit vector = Bool[0, 1, 1, 1, 0, 1, 1, 1]


In [5]:
z = simplifyLogic(Meta.parse("x1 * (1 ⊕ x1)"))

:(x1 * ~x1)

In [6]:
z.args[3].args

2-element Vector{Any}:
 :~
 :x1

In [7]:
BitArray([((i-1) >> (j-1)) & 1  for i in 1:2^3, j in 1:3])

8×3 BitMatrix:
 0  0  0
 1  0  0
 0  1  0
 1  1  0
 0  0  1
 1  0  1
 0  1  1
 1  1  1

In [8]:
simplifyLogic(Meta.parse("(z1 ⊕ z3) ⊕ (0 ⊕ z3)"))

:(z3 ⊕ (z1 ⊕ z3))

In [9]:
simplifyLogic(Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)"))

:(x2 * (x3 * (x1 + x2)))

In [10]:
simplifyLogic(Meta.parse("0 + x1"))

:x1

In [11]:
simplifyLogic(Meta.parse("x1 * (x1 + 0)")) 

:x1

In [12]:
init_logic(3)

f1 = "x1 + x2 * x3"
f2 = "(z1 * z3) ⊕ (z2 * z3)"
zero = "((x1 + x2) * x3) ⊕ (x1 * x3 + x2 * x3)"

println("\nExamining the 'f2' formula:")
f2e = create_bool_rep(f2)
println(f2e)
print("Number of non-zero bits: ")
println(logicCount(f2e))
println("First 5 (at most 5) variable values that lead to a 'true' value are:")
println("$(f2e.var)1, $(f2e.var)2, $(f2e.var)3):")
res = nonZero(f2e, head=5)
print(res)

println("\nExamining the 'zero' formula:")
zeroe = create_bool_rep(zero)
println(zeroe)
print("Number of non-zero bits: ")
println(logicCount(zeroe))
println("First 5 (at most 5) variable values that lead to a 'true' value are:")
res = nonZero(zeroe, head=5)
typeof(res)
res
logicCount(zeroe)


Examining the 'f2' formula:
Formula    = (z1 * z3) ⊕ (z2 * z3)
Variable   = z
Size       = 3
Bit vector = Bool[0, 0, 0, 0, 0, 1, 1, 0]

Number of non-zero bits: 2
First 5 (at most 5) variable values that lead to a 'true' value are:
z1, z2, z3):
(1, 0, 1)
(0, 1, 1)

Examining the 'zero' formula:
Formula    = ((x1 + x2) * x3) ⊕ (x1 * x3 + x2 * x3)
Variable   = x
Size       = 3
Bit vector = Bool[0, 0, 0, 0, 0, 0, 0, 0]

Number of non-zero bits: 0
First 5 (at most 5) variable values that lead to a 'true' value are:


0

In [13]:
create_bool_rep("(z1 + z2) * z3")

Formula    = (z1 + z2) * z3
Variable   = z
Size       = 3
Bit vector = Bool[0, 0, 0, 0, 0, 1, 1, 1]


In [14]:
isEquiv(f1, f1)

true

In [15]:
Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)")

:(((x1 + x2) * x3) * (x1 * 0 + x2 * 1))

In [16]:
simplifyLogic(Meta.parse("((x1 + x2) * x3) * (x1 * 0 + x2 * 1)"))

:(x2 * (x3 * (x1 + x2)))

In [17]:
frm="((1 + x2) * x3) * x2"
ee = simplifyLogic(Meta.parse(frm))
string(ee)

"x2 * x3"

In [18]:
    x = reverse(bitstring(UInt8(11)))

"11010000"

In [19]:
simplifyLogic(Meta.parse("x1 ⊕ 0 ⊕ x1 ⊕ 1 ⊕ x1"))

:(~x1)

### Operator Precedence: XOR lower than "*" (Both left associative)

In [20]:
simplifyLogic(Meta.parse("1 ⊕ 1 * 0"))

1

In [21]:
simplifyLogic(Meta.parse("(1 ⊕ 1) * 0"))

0

### Operator Precedence: XOR same as "+" (Both left associative)

#### Is "+" higher than XOR: NO

In [22]:
simplifyLogic(Meta.parse("1 ⊕ 1 + 1"))

1

In [23]:
simplifyLogic(Meta.parse("1 ⊕ (1 + 1)"))

0

#### Is XOR higher than "+": NO

In [24]:
simplifyLogic(Meta.parse("1 + 1 ⊕ 1"))

0

In [25]:
simplifyLogic(Meta.parse("1 + (1 ⊕ 1)"))

1

### Extending the Function nonZero
For future improvement of the package. 

In [26]:
# Intension to take a formula and find inputs that return 1.
form="(x1 + (x2 * ~x5)) * (x3 + x4) +  ~x2"

# Total number of variables.
N=5
NV=3

# Init the logic system for (N-NV) variables.
init_logic(N-NV)

x = Vector{Int64}(undef, N)

# For each of the first `NV` variables
# loop through all values of `0` and `1`s.
# For each set of NV values, substitute these into `form`
# and then simply the formula.
# Remap the variables that remain to 1 to (N-NV).

d = Dict{String,String}()
for i in (NV+1):N
    d["x$i"] = "x$(i-NV)"
end

println(d)

for i in 0:(2^NV-1)
    x = reverse(bitstring(UInt8(i)))
    println("\tx = $(x[1:N])")
    frm = form
    firstBits = x[1:NV]
    for j in 1:NV
        frm = replace(frm, "x$j" => x[j])
    end
    
    eqn = simplifyLogic(Meta.parse(frm))
    if eqn == 0 || eqn == 1
        if eqn == 1
            println(x)
        end
    else
        eqn = string(eqn)
        # We need to convert the vars over to 1:(N-NV). FIXME
        for k in keys(d)
            eqn = replace(eqn, k => d[k])
        end
        println("eqn = $eqn")
        eqnb = create_bool_rep(string(eqn))
        if logicCount(eqnb) != 0
            res = nonZero(eqnb, head=5)
            println(typeof(res))
            print(string(res))
        end
    end
end

Dict("x5" => "x2", "x4" => "x1")
	x = 00000
00000000
	x = 10000
10000000
	x = 01000
eqn = x1 * ~x2
BitMatrix
(1, 0)
	x = 11000
eqn = x1
BitMatrix
(1, 0)
(1, 1)
	x = 00100
00100000
	x = 10100
10100000
	x = 01100
eqn = ~x2
BitMatrix
(0, 0)
(1, 0)
	x = 11100
11100000


In [27]:
ex = simplifyLogic(Boolean.fixXorParseTree(Meta.parse("x1 ⊕ x2 ⊕ x3 ⊕ x4")))

:(⊕(x1, x2, x3, x4))

In [28]:
ex = Meta.parse("x1 ⊕ x2 ⊕ x3 ⊕ x4")

:(((x1 ⊕ x2) ⊕ x3) ⊕ x4)

In [29]:
ex.args[2]

:((x1 ⊕ x2) ⊕ x3)

In [30]:
z = Boolean.fixXorParseTree(ex)

:(⊕(x1, x2, x3, x4))

In [31]:
z.head

:call

In [32]:
z.args

5-element Vector{Any}:
 :⊕
 :x1
 :x2
 :x3
 :x4

In [33]:
simplifyLogic(z)

:(⊕(x1, x2, x3, x4))

In [34]:
ex.args[2].args[1]

:⊕

In [35]:
ex = Meta.parse("x1 + x1 + x1")

:(x1 + x1 + x1)

In [36]:
ex.args

4-element Vector{Any}:
 :+
 :x1
 :x1
 :x1

In [37]:
ex = Meta.parse("x1 ⊕ x2 ⊕ x3 ⊕ x4")

:(((x1 ⊕ x2) ⊕ x3) ⊕ x4)

In [38]:
z = Boolean.fixXorParseTree(ex)

:(⊕(x1, x2, x3, x4))

In [39]:
ex = Meta.parse("x1 ⊕ x2 ⊕ x3")

:((x1 ⊕ x2) ⊕ x3)

In [40]:
zz = Boolean.fixXorParseTree(ex)

:(⊕(x1, x2, x3))

In [41]:
zz.args

4-element Vector{Any}:
 :⊕
 :x1
 :x2
 :x3

In [42]:
z = Boolean.fixXorParseTree(ex)

:(⊕(x1, x2, x3))

In [43]:
z.head

:call

In [44]:
z.args

4-element Vector{Any}:
 :⊕
 :x1
 :x2
 :x3

In [45]:
ex = Meta.parse("(x1 * x2) ⊕ (x2 ⊕ x3) ⊕ (x2 + x5) ⊕ x4")

:(((x1 * x2 ⊕ (x2 ⊕ x3)) ⊕ (x2 + x5)) ⊕ x4)

In [46]:
z = Boolean.fixXorParseTree(ex)

:(⊕(x1 * x2, x2, x3, x2 + x5, x4))

In [47]:
z.args

6-element Vector{Any}:
 :⊕
 :(x1 * x2)
 :x2
 :x3
 :(x2 + x5)
 :x4

In [48]:
ex = Meta.parse("(x1 ⊕ x3) ⊕ (x1 ⊕ x3)")

:((x1 ⊕ x3) ⊕ (x1 ⊕ x3))

In [49]:
ex.args[2].args

3-element Vector{Any}:
 :⊕
 :x1
 :x3

In [50]:
simplifyLogic(ex)

0

In [51]:
ex = Meta.parse("x1 ⊕ (x2 + x3) ⊕ x4 ⊕ x5")

:(((x1 ⊕ (x2 + x3)) ⊕ x4) ⊕ x5)

In [52]:
zz = Boolean.fixXorParseTree(ex; verbose=true)

  Expr: ((x1 ⊕ (x2 + x3)) ⊕ x4) ⊕ x5
    Expr: (x1 ⊕ (x2 + x3)) ⊕ x4
      Expr: x1 ⊕ (x2 + x3)
        Symbol: x1
        Expr: x2 + x3
          Symbol: x2
          Symbol: x3
      Symbol: x4
    Symbol: x5


:(⊕(x1, x2 + x3, x4, x5))

In [53]:
zz.args

5-element Vector{Any}:
 :⊕
 :x1
 :(x2 + x3)
 :x4
 :x5

In [54]:
simplifyLogic(parseLogic("x1 ⊕ (x2 + x3) ⊕ x4 ⊕ x5"))

:(⊕(x1, x4, x5, x2 + x3))

In [55]:
simplifyLogic(Boolean.fixXorParseTree(Meta.parse("(z1 ⊕ z3) ⊕ (0 ⊕ z3)")))

:z1

In [56]:
simplifyLogic(Boolean.fixXorParseTree(Meta.parse("((1 + x2) * x3) * x2")))

:(x2 * x3)

In [57]:
using Boolean

In [58]:
simplifyLogic(parseLogic("(x1 * x2) + (x2 * x3) + (x1 * x2)"))

:(x1 * x2 + x2 * x3)

In [59]:
z = parseLogic("(x1 * x2) + (x2 * x3) + (x1 * x2)")

:(x1 * x2 + x2 * x3 + x1 * x2)

In [60]:
z.args

4-element Vector{Any}:
 :+
 :(x1 * x2)
 :(x2 * x3)
 :(x1 * x2)

In [61]:
parseLogic("x1 * x2 + x2 * x3 + x1 * x2")

:(x1 * x2 + x2 * x3 + x1 * x2)

In [62]:
parseLogic("x1 * x2 + x2 * x3 + x1 * x2 + ~x10")

:(x1 * x2 + x2 * x3 + x1 * x2 + ~x10)

In [63]:
macro bfunc(x)
    begin
    sform = string(x)
    num_vars = maximum([parse(Int64, x) for x in split(sform, r"[ *+~()⊕a-z]+") if x != ""])
    init_logic(num_vars)
    return(create_bool_rep(sform));
    end
end

@bfunc (macro with 1 method)

In [64]:
init_logic(10)

10

In [65]:
Boolean.vars

1024×10 BitMatrix:
 0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 1  0  1  0  0  0  0  0  0  0
 0  1  1  0  0  0  0  0  0  0
 1  1  1  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0
 1  0  0  1  0  0  0  0  0  0
 0  1  0  1  0  0  0  0  0  0
 1  1  0  1  0  0  0  0  0  0
 0  0  1  1  0  0  0  0  0  0
 ⋮              ⋮           
 0  0  1  0  1  1  1  1  1  1
 1  0  1  0  1  1  1  1  1  1
 0  1  1  0  1  1  1  1  1  1
 1  1  1  0  1  1  1  1  1  1
 0  0  0  1  1  1  1  1  1  1
 1  0  0  1  1  1  1  1  1  1
 0  1  0  1  1  1  1  1  1  1
 1  1  0  1  1  1  1  1  1  1
 0  0  1  1  1  1  1  1  1  1
 1  0  1  1  1  1  1  1  1  1
 0  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1

In [66]:
sizeof(Boolean.vars)

1280

In [67]:
1024 * 10 / 8

1280.0

In [68]:
form = @bfunc x1 + x2 ⊕ x3 * x2 + ~x10

Formula    = ((x1 + x2) ⊕ x3 * x2) + ~x10
Variable   = x
Size       = 10
Bit vector = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 1, 1, 1, 0, 1, 0, 0]


In [69]:
sizeof(form)

32

In [70]:
form

Formula    = ((x1 + x2) ⊕ x3 * x2) + ~x10
Variable   = x
Size       = 10
Bit vector = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 1, 1, 1, 0, 1, 0, 0]


In [71]:
sizeof(form.val)

128

In [72]:
typeof(form)

Blogic

In [73]:
form

Formula    = ((x1 + x2) ⊕ x3 * x2) + ~x10
Variable   = x
Size       = 10
Bit vector = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 1, 1, 1, 0, 1, 0, 0]


In [74]:
str = "x1 + x2 ⊕ x3 * (x2 + x4)"
xx = maximum([parse(Int64, x) for x in split(str, r"[ *+~()⊕a-z]+") if x != ""])

4

In [75]:
sizeof(form.val)

128

In [76]:
sizeof(BitVector(Bool[0,1,1,1,0,1, 0, 0,0,1,0,1,1,1,1,0,0,0,1,0,0,1,1]))

8

In [77]:
display(form)

Formula    = ((x1 + x2) ⊕ x3 * x2) + ~x10
Variable   = x
Size       = 10
Bit vector = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 1, 1, 1, 0, 1, 0, 0]


In [78]:
println(form.val)

Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,